# Analyzing the DIRNDL data set
---

This notebook contains some basic operations to analyze the portion of the DIRNDL corpus that is used to create PromDetect. The results of the operations can also be found in section [X] of the thesis. While the operations are not completely reproducible because the data is not made public, this document is supposed to still provide some insight into how the numbers in that section came to be.

In [29]:
from promdetect.prep import prepare_data
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from glob import glob
from functools import reduce

import re


In [2]:
rootDir = "/home/lukas/Dokumente/Uni/ma_thesis/"

In [3]:
# Get the paths of all recording files and extract the IDs from their filenames
corpusDir = rootDir + "quelldaten/DIRNDL-prosody"
recordingPaths = glob(pathname = corpusDir + "/*.wav")
recordingIds = [re.split(r".*/dlf-nachrichten-(.*)\.wav", rPath)[1] for rPath in recordingPaths]

In [4]:
# Run the primary data preparation function on the annotations for all recordings and store the relevant parts in separate dictionaries
accents = pd.DataFrame(columns = ["time", "label", "recording"])
tones = pd.DataFrame(columns = ["time", "label", "recording"])
transcript = pd.DataFrame(columns = ["start", "end", "label", "recording"])

for recording in recordingIds:
    currentData = prepare_data.DataPreparation(corpusDir, recording)
    currentData.transform_annotations()

    currentAccents = pd.DataFrame(currentData.accents[["time", "label"]])
    currentAccents["recording"] = recording
    accents = accents.append(currentAccents)
    
    currentTones = pd.DataFrame(currentData.tones[["time", "label"]])
    currentTones["recording"] = recording
    tones = tones.append(currentTones)
    
    currentTranscript = pd.DataFrame(currentData.transcript[["start", "end", "label"]])
    currentTranscript["recording"] = recording
    transcript = transcript.append(currentTranscript)

---
### Analyze accents

In [5]:
accents = accents.loc[~accents["label"].isna()] # drop NA-labelled rows
print("Amount of accent boundary labels: {}".format(len(accents)))

Amount of accent boundary labels: 19631


In [6]:
# Show frequency counts for each label
accents["label"].value_counts()

L*H      7819
H*L      6120
!H*L     2126
H*       2055
..L       634
L*        461
!H*       263
L*HL       53
..H        25
H*L?       18
*?         17
L*H?       13
HH*L        6
H*?         6
!H*L?       3
.L          2
LH*L        2
L*?         2
H!          1
L*!H        1
L*HL?       1
H*l         1
H*M?        1
!H          1
Name: label, dtype: int64

---
### Analyze tones

In [7]:
tones = tones.loc[~tones["label"].isna()] # drop NA-labelled rows
print("Amount of tone boundary labels: {}".format(len(tones)))

Amount of tone boundary labels: 9216


In [8]:
# Show frequency counts for each label
tones["label"].value_counts()

-      4823
%      4027
H%      173
L%      145
%H       40
-?        7
H?%       1
Name: label, dtype: int64

---
### Analyze transcripts

In [9]:
# Drop breathing sounds, paragraph markers and empty labels
transcript = transcript.loc[~transcript["label"].isin(["[@]", "[t]", "[n]", "[f]", "[h]", "<P>", np.nan])]

In [10]:
# Get the duration of each word by using its end and start timestamps
transcript["dur"] = transcript["end"] - transcript["start"]

In [11]:
# Get amount of word annotations
print("Amount of word annotation labels: {}".format(len(transcript)))

Amount of word annotation labels: 35347


In [12]:
# Get miscellaneous statistics about transcripts
print("Longest transcripts:\n{}\n".format(transcript["recording"].value_counts().head(3)))
print("Shortest transcripts:\n{}\n".format(transcript["recording"].value_counts().tail(3)))
print("Median length: {}".format(np.median(transcript["recording"].value_counts())))
print("Mean length: {}".format(np.mean(transcript["recording"].value_counts())))

Longest transcripts:
200703251200    1071
200703262000     995
200703270600     987
Name: recording, dtype: int64

Shortest transcripts:
200703251100    472
200703271100    461
200703271500    457
Name: recording, dtype: int64

Median length: 536.0
Mean length: 642.6727272727272


---
### Analyze nuclei

In [13]:
nucleiFiles = glob(rootDir + "promdetect/data/dirndl/nuclei/*")
nucleiData = pd.DataFrame(columns = ["start", "end", "timestamp_auto", "phone_label", "word_label", "origin_file"])

In [14]:
for nucleiFile in nucleiFiles:
    currentData = pd.read_csv(nucleiFile, delimiter = ",")
    currentData["origin_file"] = re.search(r"[ \w-]+?(?=\.)", nucleiFile)[0] + ".nuclei"
    nucleiData = nucleiData.append(currentData)

In [15]:
# Get miscellaneous statistics about nuclei
print("Most nuclei per recording:\n{}\n".format(nucleiData["origin_file"].value_counts().head(3)))
print("Least nuclei per recording:\n{}\n".format(nucleiData["origin_file"].value_counts().tail(3)))
print("Median nuclei per transcript: {}".format(np.median(nucleiData["origin_file"].value_counts())))
print("Mean nuclei per transcript: {}".format(np.mean(nucleiData["origin_file"].value_counts())))

Most nuclei per recording:
dlf-nachrichten-200703251200.nuclei    2084
dlf-nachrichten-200703270600.nuclei    2024
dlf-nachrichten-200703260600.nuclei    1957
Name: origin_file, dtype: int64

Least nuclei per recording:
dlf-nachrichten-200703260900.nuclei    898
dlf-nachrichten-200703271500.nuclei    884
dlf-nachrichten-200703261900.nuclei    879
Name: origin_file, dtype: int64

Median nuclei per transcript: 1032.0
Mean nuclei per transcript: 1238.8363636363636


In [16]:
# Get nuclei distances
nucleiDataGrouped = nucleiData.groupby(["origin_file"])

nucleiData["timestamp_auto_next"] = nucleiDataGrouped["timestamp_auto"].shift(-1)

nucleiData["to_next_nucleus"] = nucleiData["timestamp_auto_next"] - nucleiData["timestamp_auto"] 

In [17]:
print("Maximum space between nuclei: {0:.2f} milliseconds".format(max(nucleiData["to_next_nucleus"] * 1000)))
print("Minimum space between nuclei: {0:.2f} milliseconds".format(min(nucleiData["to_next_nucleus"] * 1000)))
print("Median space between nuclei: {0:.2f} milliseconds".format(np.nanmedian(nucleiData["to_next_nucleus"] * 1000)))
print("Mean space between nuclei: {0:.2f} milliseconds".format(np.mean(nucleiData["to_next_nucleus"] * 1000)) )

Maximum space between nuclei: 31736.50 milliseconds
Minimum space between nuclei: 41.60 milliseconds
Median space between nuclei: 194.38 milliseconds
Mean space between nuclei: 257.26 milliseconds


In [25]:
# Get speech rate indicators
endTimes = nucleiDataGrouped["end"].max().to_frame().reset_index()
startTimes = nucleiDataGrouped["start"].min().to_frame().reset_index()
nucleusCounts = nucleiData["origin_file"].value_counts().to_frame().reset_index().rename(columns = {"index": "origin_file", "origin_file": "counts"})

In [32]:
timesCounts = reduce(lambda  left,right: pd.merge(left,right,on=['origin_file'],
                                            how='outer'), [endTimes, startTimes, nucleusCounts])
timesCounts["duration"] = timesCounts["end"] - timesCounts["start"]
timesCounts["nuc_per_min"] = timesCounts["counts"] / (timesCounts["duration"] / 60)
timesCounts["nuc_per_sec"] = timesCounts["counts"] / timesCounts["duration"]

In [36]:
print("Maximum nuclei per second: {0:.2f}".format(max(timesCounts["nuc_per_sec"])))
print("Minimum nuclei per second: {0:.2f}".format(min(timesCounts["nuc_per_sec"])))
print("Median nuclei per second: {0:.2f}".format(np.nanmedian(timesCounts["nuc_per_sec"])))
print("Mean nuclei per second: {0:.2f}".format(np.mean(timesCounts["nuc_per_sec"])))

Maximum nuclei per second: 4.25
Minimum nuclei per second: 3.50
Median nuclei per second: 3.94
Mean nuclei per second: 3.92
